In [6]:
path = 'C:/Users/Paola/Downloads/ICAFIXtimeseries/'
import os 
import fnmatch
from ipywidgets import *
from IPython.display import display
from joblib import Parallel, delayed
import multiprocessing
import scipy.io as sio

In [2]:
"""
Partial Correlation in Python (clone of Matlab's partialcorr)

This uses the linear regression approach to compute the partial 
correlation (might be slow for a huge number of variables). The 
algorithm is detailed here:

    http://en.wikipedia.org/wiki/Partial_correlation#Using_linear_regression

Taking X and Y two variables of interest and Z the matrix with all the variable minus {X, Y},
the algorithm can be summarized as

    1) perform a normal linear least-squares regression with X as the target and Z as the predictor
    2) calculate the residuals in Step #1
    3) perform a normal linear least-squares regression with Y as the target and Z as the predictor
    4) calculate the residuals in Step #3
    5) calculate the correlation coefficient between the residuals from Steps #2 and #4; 

    The result is the partial correlation between X and Y while controlling for the effect of Z


Date: Nov 2014
Author: Fabian Pedregosa-Izquierdo, f@bianp.net
Testing: Valentina Borghesani, valentinaborghesani@gmail.com
"""

import numpy as np
from scipy import stats, linalg

def partial_corr(C):
    """
    Returns the sample linear partial correlation coefficients between pairs of variables in C, controlling 
    for the remaining variables in C.


    Parameters
    ----------
    C : array-like, shape (n, p)
        Array with the different variables. Each column of C is taken as a variable


    Returns
    -------
    P : array-like, shape (p, p)
        P[i, j] contains the partial correlation of C[:, i] and C[:, j] controlling
        for the remaining variables in C.
    """
    
    C = np.asarray(C)
    p = C.shape[1]
    P_corr = np.zeros((p, p), dtype=np.float)
    for i in range(p):
        P_corr[i, i] = 1
        for j in range(i+1, p):
            idx = np.ones(p, dtype=np.bool)
            idx[i] = False
            idx[j] = False
            beta_i = linalg.lstsq(C[:, idx], C[:, j])[0]
            beta_j = linalg.lstsq(C[:, idx], C[:, i])[0]

            res_j = C[:, j] - C[:, idx].dot( beta_i)
            res_i = C[:, i] - C[:, idx].dot(beta_j)
            
            corr = stats.pearsonr(res_i, res_j)[0]
            P_corr[i, j] = corr
            P_corr[j, i] = corr
        
    return P_corr

In [ ]:
files = os.listdir(path+'rfMRI_REST1')
l = int(len(files)/2)


def processInput(fn,regex,folder):
    if fnmatch.fnmatch(fn, regex):
        ts = np.loadtxt(path+folder+fn)
        pcorr = partial_corr(ts)
        return pcorr

num_cores = multiprocessing.cpu_count()
    
results = Parallel(n_jobs=num_cores,verbose=1)(delayed(processInput)(fn,'*LR.txt','rfMRI_REST1/') for fn in files)

In [3]:
files = os.listdir(path+'rfMRI_REST1')
l = int(len(files)/2)

pcorr_LR_1 = np.zeros([268,268,l])
i=0
f = FloatProgress(min=0, max=l)
display(f)
for fn in files:
     if fnmatch.fnmatch(fn, '*LR.txt'):
        ts = np.loadtxt(path+'rfMRI_REST1/'+fn)
        pcorr_LR_1[:,:,i] = partial_corr(ts)
        f.value += 1
        i = i+1
        

KeyboardInterrupt: 

In [4]:
print(pcorr_LR_1[:,:,0])

[[ 1.         -0.0295215  -0.02509423 ..., -0.02757572 -0.00985024
  -0.02619222]
 [-0.0295215   1.          0.0017804  ...,  0.00444317 -0.03650057
   0.07437129]
 [-0.02509423  0.0017804   1.         ..., -0.01888155  0.05945833
  -0.03038365]
 ..., 
 [-0.02757572  0.00444317 -0.01888155 ...,  1.         -0.07551081
   0.05016466]
 [-0.00985024 -0.03650057  0.05945833 ..., -0.07551081  1.          0.0748843 ]
 [-0.02619222  0.07437129 -0.03038365 ...,  0.05016466  0.0748843   1.        ]]


In [7]:
results = {'pcorr_LR_1':pcorr_LR_1}
sio.savemat('pcorr_LR_1_intermediate.mat',results)